# 📊 Preparación del Conjunto de Datos
En este notebook se desarrolla el proceso de **feature engineering** destinado a la **construcción del conjunto de datos histórico de la criptomoneda Bitcoin (BTC) frente al dólar estadounidense (USD)**. A partir de los datos originales, se generan y seleccionan indicadores técnicos y variables adicionales que enriquecen la representación de la serie temporal. El resultado es un dataset estructurado y preparado para su utilización en fases posteriores de entrenamiento y modelización mediante redes neuronales, cuyo propósito es la **predicción del movimiento diario del precio de BTC/USD**.

In [1]:
# =============================================================================
# LIBRERIAS
# =============================================================================

import pandas as pd
import numpy as np
import ta
import requests
from pathlib import Path
import os

In [2]:
# =============================================================================
# CONSTANTES
# =============================================================================

BASE_DIR = Path().resolve() # Carpeta base (donde está el notebook)
BLOCKCHAIN_PATH = "data/blockchain" # Carpeta con datos de la blockchain
BLOCKCHAIN_DAILY_CSV = "bdc_daily_data.csv" #

## 🔹 Importación de Datos

En esta sección importaremos los datos históricos de Bitcoin frente al dólar estadounidense (BTC/USD).  
Los datos han sido obtenidos de [Kaggle: Comprehensive BTC/USD 1m Data](https://www.kaggle.com/datasets/imranbukhari/comprehensive-btcusd-1m-data/data).

Para nuestro análisis utilizaremos el archivo **`BTCUSD_1m_Binance.csv`** que, aunque cubre un período más corto (desde 2017), incluye **variables adicionales** como `Number of trades`, `Taker buy base asset volume` y `Taker buy quote asset volume`, las cuales podrían resultar útiles para la predicción del movimiento diario del precio de BTC/USD. Además, al centrarnos en un período más reciente, este conjunto de datos **refleja mejor el comportamiento actual del mercado**, evitando introducir ruido o patrones históricos menos relevantes para nuestras predicciones en la actualidad.

In [3]:
# =============================================================================
# LECTURA CSV
# =============================================================================

df_binance_1m = pd.read_csv("C:/Users/misan/OneDrive - Universidad Complutense de Madrid (UCM)/TFM/dataset/BTCUSD_1m_Binance.csv")
df_binance_1m

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.775183,2017-08-17 04:00:59.999,7564.906851,3.0,0.075183,320.390851,0.0
1,2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.000000,2017-08-17 04:01:59.999,0.000000,0.0,0.000000,0.000000,0.0
2,2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074,2017-08-17 04:02:59.999,1117.542921,2.0,0.261074,1117.542921,0.0
3,2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.012008,2017-08-17 04:03:59.999,51.171852,3.0,0.012008,51.171852,0.0
4,2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.140796,2017-08-17 04:04:59.999,599.999338,1.0,0.140796,599.999338,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4205248,2025-08-21 11:20:00,113499.64,113499.65,113461.58,113461.58,3.008920,2025-08-21 11:20:59.999,341477.818531,1805.0,0.331930,37663.813893,0.0
4205249,2025-08-21 11:21:00,113461.58,113461.58,113433.83,113433.84,1.349480,2025-08-21 11:21:59.999,153099.233618,1033.0,0.305090,34613.711661,0.0
4205250,2025-08-21 11:22:00,113433.84,113482.97,113433.83,113482.97,5.305390,2025-08-21 11:22:59.999,601925.713161,1155.0,4.921030,558319.716437,0.0
4205251,2025-08-21 11:23:00,113482.97,113489.66,113464.00,113489.66,5.430800,2025-08-21 11:23:59.999,616278.382456,1199.0,2.278360,258536.719165,0.0


### 📌 Descripción de las Columnas del Dataset

A continuación se detallan las columnas presentes en el archivo **`BTCUSD_1m_Binance.csv`**:

- **`Open time`**: Timestamp de apertura de la vela. Indica el momento exacto en que comenzó el intervalo de tiempo de la vela.
- **`Open`**: Precio de apertura de la vela. Es el primer precio registrado en ese intervalo.
- **`High`**: Precio máximo alcanzado durante la vela.
- **`Low`**: Precio mínimo alcanzado durante la vela.
- **`Close`**: Precio de cierre de la vela. Es el último precio registrado en ese intervalo.
- **`Volume`**: Cantidad de BTC negociada durante la vela.
- **`Close time`**: Timestamp de cierre de la vela. Representa el final del intervalo de la vela.
- **`Quote asset volume`**: Volumen negociado en la moneda cotizada (por ejemplo, USD) durante la vela.
- **`Number of trades`**: Número total de operaciones realizadas durante la vela.
- **`Taker buy base asset volume`**: Volumen de BTC comprado por los **takers** (operadores que aceptan el precio de mercado) durante la vela.
- **`Taker buy quote asset volume`**: Volumen en la moneda cotizada comprado por los **takers**.
- **`Ignore`**: Columna reservada por Binance, generalmente sin uso (valores nulos o cero).

## 🔹 Procesado

En esta sección realizaremos el **procesado de los datos** para preparar el dataset para su análisis y modelización.  
Durante este paso, **generaremos nuevas variables derivadas** que aporten información adicional sobre el comportamiento del mercado y de la blockchain, lo que nos permitirá identificar las más relevantes para la predicción y mejorar la precisión de nuestro modelo.

### 📌 Columnas descartadas

Durante el procesado, **descartaremos las columnas `Ignore` y `Close time`**:  

- **`Ignore`**: columna reservada por Binance que no contiene información útil (valores nulos o siempre cero).  
- **`Close time`**: timestamp redundante, ya que se puede derivar directamente de `Open time` y la duración de la vela; no aporta valor adicional para nuestra predicción diaria.

### 📌 Agregación Diaria

Nuestro objetivo es **predecir si el precio de BTC/USD subirá o bajará al día siguiente**, por lo que necesitamos trabajar con datos agregados a nivel diario.  
Para ello, transformaremos los datos originales de 1 minuto a **velas diarias**, calculando:  

- `Open`: primer precio del día  
- `High`: precio máximo del día  
- `Low`: precio mínimo del día  
- `Close`: precio de cierre del día  
- `Volume`: volumen total negociado en BTC  
- `Quote asset volume`: volumen total en moneda cotizada  
- Otras variables relevantes (`Number of trades`, `Taker buy ...`) agregadas por día  

Esta agregación diaria nos permite **resumir la información relevante del mercado** y preparar el dataset de manera coherente con nuestro objetivo de predicción diaria.

In [4]:
# =============================================================================
# AGREGACION DIARIA
# =============================================================================

# Convertimos la columna 'Open time' a tipo datetime para poder manipular fechas
df_binance_1m['Open time'] = pd.to_datetime(df_binance_1m['Open time'])

# Ponemos 'Open time' como índice temporal para usar resample
df_binance_1m.set_index('Open time', inplace=True)

# Reagrupar a diario usando 'resample' y aplicar distintas funciones según la columna
df_binance_1d = df_binance_1m.resample('1D').agg({
    'Open': 'first',                      # Primer precio del día
    'High': 'max',                        # Precio máximo del día
    'Low': 'min',                         # Precio mínimo del día
    'Close': 'last',                      # Último precio del día
    'Volume': 'sum',                       # Suma del volumen de todas las velas del día
    'Quote asset volume': 'sum',           # Suma del volumen en la moneda cotizada
    'Number of trades': 'sum',             # Suma del número de trades diarios
    'Taker buy base asset volume': 'sum',  # Suma del volumen comprado por takers
    'Taker buy quote asset volume': 'sum'  # Suma de la moneda cotizada comprada por takers
})

# Resetear el índice para que 'Open time' vuelva a ser columna
df_binance_1d = df_binance_1d.reset_index()

# Eliminamos el ultimo día
df_binance_1d = df_binance_1d.iloc[:-1]

# Guardamos el dataframe diario resultante en un nuevo CSV
#df_binance_1d.to_csv("datos_binance_diario.csv", index=False)

df_binance_1d.head()

,Open time,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,3.454770e+06,3427.0,616.248541,2.678216e+06
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,5.086958e+06,5233.0,972.868710,4.129123e+06
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,1.549484e+06,2153.0,274.336042,1.118002e+06
3,2017-08-20,4139.98,4211.08,4032.62,4086.29,467.083022,1.930364e+06,2321.0,376.795947,1.557401e+06
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,2.797232e+06,3972.0,557.356107,2.255663e+06


### 📌 Creación de la Variable Objetivo: Dirección del Precio

Siguiendo la metodología propuesta por Livieris en su artículo *"An Advanced CNN-LSTM Model for Cryptocurrency Forecasting"* [(MDPI, 2021)](https://www.mdpi.com/2079-9292/10/3/287), consideramos **movimientos direccionales del precio** en lugar de la magnitud del mismo. De este modo, nuestro problema se convierte en un **problema de clasificación binaria**:

- **Clase 1**: El precio de Bitcoin aumenta del día `t` al día `t+1` (`Close_t+1 > Close_t`) y se codifica como `1`.
- **Clase 0**: El precio de Bitcoin disminuye o se mantiene igual (`Close_t+1 ≤ Close_t`) y se codifica como `0`.


Este enfoque también ha sido aplicado y comparado empíricamente en estudios como *"Deep learning for Bitcoin price direction prediction: models and trading strategies empirically compared"* de Oluwadamilare Omole y David Enke [(Springer, 2024)](https://jfin-swufe.springeropen.com/articles/10.1186/s40854-024-00643-1).

In [5]:
# =============================================================================
# VARIABLE OBJETIVO
# =============================================================================

# Creamos la variable objetivo 'Target' según el movimiento direccional del precio
df_binance_1d['Target'] = (df_binance_1d['Close'].shift(-1) > df_binance_1d['Close']).astype(int)

# Revisamos las primeras filas
df_binance_1d[['Close', 'Target']].head(10)

,Close,Target
0,4285.08,0
1,4108.37,1
2,4139.98,0
3,4086.29,0
4,4016.00,1
5,4040.00,1
6,4114.01,1
7,4316.01,0
8,4280.68,1
9,4337.44,0


In [6]:
df_binance_1d

,Open time,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Target
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,3.454770e+06,3427.0,616.248541,2.678216e+06,0
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,5.086958e+06,5233.0,972.868710,4.129123e+06,1
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,1.549484e+06,2153.0,274.336042,1.118002e+06,0
3,2017-08-20,4139.98,4211.08,4032.62,4086.29,467.083022,1.930364e+06,2321.0,376.795947,1.557401e+06,0
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,2.797232e+06,3972.0,557.356107,2.255663e+06,1
...,...,...,...,...,...,...,...,...,...,...,...
2921,2025-08-16,117342.04,117898.99,117143.98,117380.66,6393.681170,7.517194e+08,1179842.0,2995.228650,3.521588e+08,1
2922,2025-08-17,117380.66,118575.00,117172.21,117405.01,5898.641920,6.956292e+08,1177563.0,2804.731130,3.307994e+08,0
2923,2025-08-18,117405.01,117543.75,114640.14,116227.05,17741.469250,2.053300e+09,3345487.0,7647.218200,8.850528e+08,0
2924,2025-08-19,116227.05,116725.69,112732.58,112872.94,18065.420860,2.065005e+09,3291170.0,8609.360780,9.840874e+08,1


## 🔹 Indicadores Técnicos

En esta sección añadiremos **indicadores técnicos** al dataset diario de BTC/USD.  
Estos indicadores aportan información adicional sobre la **tendencia, momentum, volatilidad y volumen del mercado**, lo que puede mejorar la capacidad predictiva de nuestros modelos.

### 📈 Indicadores de Tendencia

Estos indicadores ayudan a identificar la dirección general del mercado y posibles cambios de tendencia. Para la predicción diaria hemos seleccionado los siguientes:

- **SMA (Simple Moving Average)**: medias móviles simples de **20 y 50 días**, que representan tendencias de medio y largo plazo.
- **EMA (Exponential Moving Average)**: medias móviles exponenciales de **20 y 50 días**, más sensibles a los cambios recientes de precio.
- **MACD (Moving Average Convergence Divergence)**: con parámetros clásicos **12, 26, 9**, útil para identificar cambios de impulso en la tendencia.
- **Parabolic SAR**: indicador de stop and reverse, que señala posibles puntos de cambio de dirección de la tendencia.
- **KAMA (Kaufman Adaptive Moving Average)**: media móvil adaptativa con **n=10 días**, que ajusta su suavizado según la volatilidad reciente, capturando cambios rápidos de tendencia a corto plazo.
- **Ratios Close / Indicador**: Para SMA, EMA y KAMA calculamos además el **ratio del precio de cierre respecto al indicador**. Estos ratios reflejan si el precio está por encima o por debajo de su media, capturando la fuerza relativa de la tendencia y normalizando los indicadores para mejorar la predicción.

#### 📝 Ventanas Seleccionadas

Para la **predicción diaria** de BTC/USD, las ventanas más importantes para indicadores de tendencia suelen ser las **medias y largas**, ya que:

- Capturan la tendencia general del mercado.
- Proporcionan contexto sobre la dirección del precio.
- Evitan que el modelo se sobreajuste a fluctuaciones de muy corto plazo.

👉 Las ventanas **cortas** pueden aportar algo de sensibilidad a cambios recientes, pero su utilidad es limitada y, si se usan en exceso, pueden introducir **ruido** en la predicción. Debido a ello, en este caso, predominarán las **ventanas medias y largas** en los indicadores técnicos seleccionado. Esto asegura que los indicadores capturen la tendencia diaria relevante sin introducir demasiado ruido de corto plazo.

In [7]:
# =============================================================================
# INDICADORES DE TENDENCIA
# =============================================================================

# ----------------------
# SMA: medio plazo
# ----------------------
for w in [20, 50]:
    df_binance_1d[f'SMA_{w}'] = df_binance_1d['Close'].rolling(window=w).mean()
    df_binance_1d[f'Close_SMA_ratio_{w}'] = df_binance_1d['Close'] / df_binance_1d[f'SMA_{w}']

# ----------------------
# EMA: medio plazo
# ----------------------
for w in [20, 50]:
    df_binance_1d[f'EMA_{w}'] = df_binance_1d['Close'].ewm(span=w, adjust=False).mean()
    df_binance_1d[f'Close_EMA_ratio_{w}'] = df_binance_1d['Close'] / df_binance_1d[f'EMA_{w}']

# ----------------------
# MACD clásico (12,26,9)
# ----------------------
macd = ta.trend.MACD(close=df_binance_1d['Close'], window_slow=26, window_fast=12, window_sign=9)
df_binance_1d['MACD'] = macd.macd()
df_binance_1d['MACD_signal'] = macd.macd_signal()
df_binance_1d['MACD_diff'] = macd.macd_diff()

# ----------------------
# Parabolic SAR
# ----------------------
df_binance_1d['PSAR'] = ta.trend.PSARIndicator(
    high=df_binance_1d['High'],
    low=df_binance_1d['Low'],
    close=df_binance_1d['Close'],
    step=0.02,
    max_step=0.2
).psar()

# ----------------------
# KAMA: calculado manualmente
# ----------------------
def KAMA(close, n=10, fast=2, slow=30):
    close = close.values
    kama = np.zeros_like(close)
    kama[:n] = close[:n].mean()  # inicializamos con la media de los primeros n valores
    
    fastest = 2 / (fast + 1)
    slowest = 2 / (slow + 1)
    
    for i in range(n, len(close)):
        change = abs(close[i] - close[i - n])
        volatility = np.sum(np.abs(close[i - n + 1:i + 1] - close[i - n:i]))
        ER = 0 if volatility == 0 else change / volatility
        SC = (ER * (fastest - slowest) + slowest) ** 2
        kama[i] = kama[i-1] + SC * (close[i] - kama[i-1])
    
    return pd.Series(kama, index=df_binance_1d.index)

df_binance_1d['KAMA_10'] = KAMA(df_binance_1d['Close'], n=10)
df_binance_1d['Close_KAMA_ratio'] = df_binance_1d['Close'] / df_binance_1d['KAMA_10']

df_binance_1d

,Open time,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,...,EMA_20,Close_EMA_ratio_20,EMA_50,Close_EMA_ratio_50,MACD,MACD_signal,MACD_diff,PSAR,KAMA_10,Close_KAMA_ratio
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,3.454770e+06,3427.0,616.248541,2.678216e+06,...,4285.080000,1.000000,4285.080000,1.000000,NaN,NaN,NaN,4285.080000,4172.386000,1.027009
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,5.086958e+06,5233.0,972.868710,4.129123e+06,...,4268.250476,0.962542,4278.150196,0.960315,NaN,NaN,NaN,4108.370000,4172.386000,0.984657
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,1.549484e+06,2153.0,274.336042,1.118002e+06,...,4256.034240,0.972732,4272.731757,0.968930,NaN,NaN,NaN,4485.390000,4172.386000,0.992233
3,2017-08-20,4139.98,4211.08,4032.62,4086.29,467.083022,1.930364e+06,2321.0,376.795947,1.557401e+06,...,4239.868122,0.963778,4265.420316,0.958004,NaN,NaN,NaN,4472.682200,4172.386000,0.979365
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,2.797232e+06,3972.0,557.356107,2.255663e+06,...,4218.547349,0.951986,4255.639127,0.943689,NaN,NaN,NaN,4460.228556,4172.386000,0.962519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2921,2025-08-16,117342.04,117898.99,117143.98,117380.66,6393.681170,7.517194e+08,1179842.0,2995.228650,3.521588e+08,...,117500.038398,0.998984,114903.399032,1.021560,936.288157,933.843987,2.444170,124474.000000,118059.623964,0.994249
2922,2025-08-17,117380.66,118575.00,117172.21,117405.01,5898.641920,6.956292e+08,1177563.0,2804.731130,3.307994e+08,...,117490.988074,0.999268,115001.501423,1.020900,818.663667,910.807923,-92.144256,124320.599800,118056.668853,0.994480
2923,2025-08-18,117405.01,117543.75,114640.14,116227.05,17741.469250,2.053300e+09,3345487.0,7647.218200,8.850528e+08,...,117370.613020,0.990257,115049.562152,1.010235,623.209908,853.288320,-230.078412,124170.267604,118044.386618,0.984605
2924,2025-08-19,116227.05,116725.69,112732.58,112872.94,18065.420860,2.065005e+09,3291170.0,8609.360780,9.840874e+08,...,116942.263208,0.965202,114964.204420,0.981809,195.410184,721.712693,-526.302509,123789.062500,117875.726041,0.957559


### ⚡ Indicadores de Momentum

Estos indicadores ayudan a identificar la **velocidad y fuerza de los movimientos de precio**, capturando sobrecompra, sobreventa y aceleraciones del mercado. Para la predicción diaria hemos seleccionado los siguientes:

- **RSI (Relative Strength Index)**: con ventanas de **7 y 14 días**, mide la fuerza relativa de las subidas y bajadas recientes.  
  - 7 días → captura movimientos rápidos del precio.  
  - 14 días → refleja momentum de corto a medio plazo.
- **Williams %R**: con ventanas de **7 y 14 días**, indica condiciones de sobrecompra o sobreventa de manera similar al RSI, pero en escala -100 a 0.  
- **Stochastic Oscillator (K/D)**: con ventanas **K=7, D=3 y K=14, D=3**, compara el precio de cierre respecto al rango reciente, mostrando si el activo está cerca de máximos o mínimos recientes.  
  - K rápido → sensibilidad a cambios recientes.  
  - D suaviza el ruido del K.  
- **ROC (Rate of Change)**: con ventanas **5, 10 y 14 días**, mide la variación porcentual del precio respecto a n días atrás.  
  - 5 días → momentum muy reciente.  
  - 10–14 días → captura impulso de 1–2 semanas.  
- **Momentum simple**: diferencia entre el precio actual y el precio n días atrás, con ventanas **5 y 10 días**, útil para reflejar el impulso inmediato del mercado.
- **Cambios diarios del momentum**: derivadas de RSI, ROC y Momentum simple, que capturan **aceleraciones** y **desaceleraciones** del impulso.

#### 📝 Ventanas Seleccionadas

Para la **predicción diaria**, las ventanas más útiles para indicadores de momentum suelen ser **cortas a medias**, ya que:

- Capturan movimientos recientes y aceleraciones del precio.  
- Detectan señales tempranas de cambio en la dirección del mercado.  
- Permiten al modelo reaccionar a sobrecompra o sobreventa en escala diaria.  

👉 Las ventanas demasiado largas tienden a **suavizar demasiado** el momentum, perdiendo sensibilidad para predicción diaria. Por ello, en este caso, predominarán las **ventanas cortas y medias** para capturar los movimientos diarios más relevantes sin introducir demasiado retraso.


In [8]:
# =============================================================================
# INDICADORES DE MOMENTUM
# =============================================================================

# ----------------------
# RSI
# ----------------------
df_binance_1d['RSI_7'] = ta.momentum.RSIIndicator(df_binance_1d['Close'], window=7).rsi()
df_binance_1d['RSI_14'] = ta.momentum.RSIIndicator(df_binance_1d['Close'], window=14).rsi()

# ----------------------
# Williams %R
# ----------------------
df_binance_1d['WR_7'] = ta.momentum.WilliamsRIndicator(df_binance_1d['High'], df_binance_1d['Low'], df_binance_1d['Close'], lbp=7).williams_r()
df_binance_1d['WR_14'] = ta.momentum.WilliamsRIndicator(df_binance_1d['High'], df_binance_1d['Low'], df_binance_1d['Close'], lbp=14).williams_r()

# ----------------------
# Stochastic Oscillator K/D
# ----------------------
stoch_7 = ta.momentum.StochasticOscillator(df_binance_1d['High'], df_binance_1d['Low'], df_binance_1d['Close'], window=7, smooth_window=3)
stoch_14 = ta.momentum.StochasticOscillator(df_binance_1d['High'], df_binance_1d['Low'], df_binance_1d['Close'], window=14, smooth_window=3)

df_binance_1d['STOCH_K_7'] = stoch_7.stoch()
df_binance_1d['STOCH_D_7'] = stoch_7.stoch_signal()
df_binance_1d['STOCH_K_14'] = stoch_14.stoch()
df_binance_1d['STOCH_D_14'] = stoch_14.stoch_signal()

# ----------------------
# Rate of Change (ROC)
# ----------------------
df_binance_1d['ROC_5'] = ta.momentum.ROCIndicator(df_binance_1d['Close'], window=5).roc()
df_binance_1d['ROC_10'] = ta.momentum.ROCIndicator(df_binance_1d['Close'], window=10).roc()
df_binance_1d['ROC_14'] = ta.momentum.ROCIndicator(df_binance_1d['Close'], window=14).roc()

# ----------------------
# Momentum simple = precio actual - precio n días atrás
# ----------------------
df_binance_1d['MOM_5'] = df_binance_1d['Close'] - df_binance_1d['Close'].shift(5)
df_binance_1d['MOM_10'] = df_binance_1d['Close'] - df_binance_1d['Close'].shift(10)


# ----------------------
# Cambios diarios (momentum relativo)
# ----------------------
df_binance_1d['dRSI_7'] = df_binance_1d['RSI_7'].diff()
df_binance_1d['dRSI_14'] = df_binance_1d['RSI_14'].diff()
df_binance_1d['dROC_5'] = df_binance_1d['ROC_5'].diff()
df_binance_1d['dROC_10'] = df_binance_1d['ROC_10'].diff()
df_binance_1d['dROC_14'] = df_binance_1d['ROC_14'].diff()
df_binance_1d['dMOM_5'] = df_binance_1d['MOM_5'].diff()
df_binance_1d['dMOM_10'] = df_binance_1d['MOM_10'].diff()

df_binance_1d

,Open time,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,...,ROC_14,MOM_5,MOM_10,dRSI_7,dRSI_14,dROC_5,dROC_10,dROC_14,dMOM_5,dMOM_10
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,3.454770e+06,3427.0,616.248541,2.678216e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,5.086958e+06,5233.0,972.868710,4.129123e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,1.549484e+06,2153.0,274.336042,1.118002e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-08-20,4139.98,4211.08,4032.62,4086.29,467.083022,1.930364e+06,2321.0,376.795947,1.557401e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,2.797232e+06,3972.0,557.356107,2.255663e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2921,2025-08-16,117342.04,117898.99,117143.98,117380.66,6393.681170,7.517194e+08,1179842.0,2995.228650,3.521588e+08,...,4.295395,-1305.34,2388.39,0.181972,0.093243,0.536433,-0.737603,0.725939,646.62,-823.91
2922,2025-08-17,117380.66,118575.00,117172.21,117405.01,5898.641920,6.956292e+08,1177563.0,2804.731130,3.307994e+08,...,2.798567,-2729.07,-67.00,0.133096,0.063127,-1.171860,-2.134035,-1.496828,-1423.73,-2455.39
2923,2025-08-18,117405.01,117543.75,114640.14,116227.05,17741.469250,2.053300e+09,3345487.0,7647.218200,8.850528e+08,...,1.018660,-7079.38,-447.69,-5.868641,-3.213786,-3.469603,-0.326673,-1.779907,-4350.31,-380.69
2924,2025-08-19,116227.05,116725.69,112732.58,112872.94,18065.420860,2.065005e+09,3291170.0,8609.360780,9.840874e+08,...,-1.101212,-5422.15,-3589.31,-12.082776,-7.741278,1.157710,-2.698244,-2.119872,1657.23,-3141.62


### 🔔 Indicadores de Volatilidad

Estos indicadores ayudan a medir **la amplitud y los cambios del mercado**, identificando períodos de alta o baja volatilidad que pueden preceder a movimientos significativos de precio. Para la predicción diaria hemos seleccionado los siguientes:  

- **Bollinger Bands (BB)**: bandas superiores e inferiores alrededor de una media móvil (10 y 20 días, desviación 2). Capturan **rupturas rápidas** y el rango de volatilidad reciente.  
- **Average True Range (ATR)**: indicador clásico de volatilidad (14 y 20 días), mide la amplitud promedio de los movimientos diarios. Útil para ajustar expectativas de rango de precio.  
- **Keltner Channels (KC)**: bandas basadas en ATR alrededor de una EMA (10 y 20 días, ATR interno 10). Ayudan a detectar compresiones (“squeezes”) y expansiones de volatilidad.  
- **Donchian Channels (DC)**: canal basado en máximos y mínimos de 20 días, que resalta **soportes y resistencias recientes** y el rango típico de mercado.  

#### 📝 Ventanas Seleccionadas

- Para predicción **diaria**, las ventanas **cortas y medias** permiten capturar movimientos recientes sin introducir demasiado ruido.  
- Bandas y canales más **cortos** (BB y KC 10 días) son sensibles a rupturas rápidas.  
- Bandas y canales **medios** (BB y KC 20 días, ATR 20, DC 20) reflejan el rango y volatilidad de 2-4 semanas, proporcionando contexto para el modelo.
    
👉 Por lo tanto, combinar varias ventanas de volatilidad permitirá al modelo detectar **momentos de expansión o compresión de mercado**, identificar rupturas y anticipar posibles movimientos significativos del precio en la predicción diaria.

In [9]:
# =============================================================================
# INDICADORES DE VOLATILIDAD
# =============================================================================

# ----------------------
# Bollinger Bands: 10 y 20 días
# ----------------------
for w in [10, 20]:
    bb = ta.volatility.BollingerBands(close=df_binance_1d['Close'], window=w, window_dev=2)
    df_binance_1d[f'BB_high_{w}'] = bb.bollinger_hband()
    df_binance_1d[f'BB_low_{w}'] = bb.bollinger_lband()
    df_binance_1d[f'BB_mid_{w}'] = bb.bollinger_mavg()
    df_binance_1d[f'BB_width_{w}'] = bb.bollinger_hband() - bb.bollinger_lband()
    df_binance_1d[f'BB_percent_{w}'] = bb.bollinger_pband()

# ----------------------
# ATR: 14 y 20 días
# ----------------------
for w in [14, 20]:
    atr = ta.volatility.AverageTrueRange(high=df_binance_1d['High'],
                                         low=df_binance_1d['Low'],
                                         close=df_binance_1d['Close'],
                                         window=w)
    df_binance_1d[f'ATR_{w}'] = atr.average_true_range()

# ----------------------
# Keltner Channels: 10 y 20 días (ATR interno 10)
# ----------------------
for w in [10, 20]:
    kc = ta.volatility.KeltnerChannel(high=df_binance_1d['High'],
                                      low=df_binance_1d['Low'],
                                      close=df_binance_1d['Close'],
                                      window=w,
                                      window_atr=10)
    df_binance_1d[f'KC_high_{w}'] = kc.keltner_channel_hband()
    df_binance_1d[f'KC_low_{w}'] = kc.keltner_channel_lband()
    df_binance_1d[f'KC_mid_{w}'] = kc.keltner_channel_mband()
    df_binance_1d[f'KC_width_{w}'] = kc.keltner_channel_hband() - kc.keltner_channel_lband()

# ----------------------
# Donchian Channel: 20 días
# ----------------------
dc = ta.volatility.DonchianChannel(high=df_binance_1d['High'],
                                   low=df_binance_1d['Low'],
                                   close=df_binance_1d['Close'],
                                   window=20)
df_binance_1d['DC_high_20'] = dc.donchian_channel_hband()
df_binance_1d['DC_low_20'] = dc.donchian_channel_lband()
df_binance_1d['DC_mid_20'] = dc.donchian_channel_mband()
df_binance_1d['DC_width_20'] = dc.donchian_channel_hband() - dc.donchian_channel_lband()

df_binance_1d

,Open time,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,...,KC_mid_10,KC_width_10,KC_high_20,KC_low_20,KC_mid_20,KC_width_20,DC_high_20,DC_low_20,DC_mid_20,DC_width_20
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,3.454770e+06,3427.0,616.248541,2.678216e+06,...,NaN,569.300000,4608.386667,4039.086667,NaN,569.300000,NaN,NaN,NaN,NaN
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,5.086958e+06,5233.0,972.868710,4.129123e+06,...,NaN,717.400000,4590.345000,3872.945000,NaN,717.400000,NaN,NaN,NaN,NaN
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,1.549484e+06,2153.0,274.336042,1.118002e+06,...,NaN,701.393333,4524.534444,3823.141111,NaN,701.393333,NaN,NaN,NaN,NaN
3,2017-08-20,4139.98,4211.08,4032.62,4086.29,467.083022,1.930364e+06,2321.0,376.795947,1.557401e+06,...,NaN,615.275000,4465.515000,3850.240000,NaN,615.275000,NaN,NaN,NaN,NaN
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,2.797232e+06,3972.0,557.356107,2.255663e+06,...,NaN,575.352000,4417.138667,3841.786667,NaN,575.352000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2921,2025-08-16,117342.04,117898.99,117143.98,117380.66,6393.681170,7.517194e+08,1179842.0,2995.228650,3.521588e+08,...,118489.153667,6243.882000,119839.719500,114134.349500,116987.034500,5705.370000,124474.0,111920.0,118197.0,12554.0
2922,2025-08-17,117380.66,118575.00,117172.21,117405.01,5898.641920,6.956292e+08,1177563.0,2804.731130,3.307994e+08,...,118615.827333,5852.040000,119755.607333,114147.208333,116951.407833,5608.399000,124474.0,111920.0,118197.0,12554.0
2923,2025-08-18,117405.01,117543.75,114640.14,116227.05,17741.469250,2.053300e+09,3345487.0,7647.218200,8.850528e+08,...,118556.743667,6082.504000,119688.591833,114022.092833,116855.342333,5666.499000,124474.0,111920.0,118197.0,12554.0
2924,2025-08-19,116227.05,116725.69,112732.58,112872.94,18065.420860,2.065005e+09,3291170.0,8609.360780,9.840874e+08,...,118277.603000,6552.142000,119570.170167,113803.937167,116687.053667,5766.233000,124474.0,111920.0,118197.0,12554.0


### 💹 Indicadores de Volumen  

Los indicadores de volumen permiten analizar **la participación del mercado y la presión compradora/vendedora**, ofreciendo pistas sobre la fortaleza detrás de los movimientos de precio. Para la predicción diaria hemos seleccionado los siguientes:  

- **On-Balance Volume (OBV)**: mide la presión acumulada de compra y venta a partir de los cambios en el precio y el volumen. Sus **derivadas y medias móviles** ayudan a detectar divergencias entre precio y volumen.  
- **Chaikin Money Flow (CMF, 20 días)**: relaciona precios y volumen para estimar el flujo de capital hacia dentro o fuera del activo. Un valor positivo indica **acumulación**, mientras que uno negativo refleja **distribución**.  
- **Money Flow Index (MFI, 14 días)**: combina precio y volumen, similar al RSI pero con “peso” de volumen. Permite detectar **zonas de sobrecompra o sobreventa con mayor precisión**.  
- **Accumulation/Distribution Line (ADL)**: mide la presión de compra/venta acumulada, evaluando si los movimientos de volumen respaldan los cambios de precio. Su pendiente refleja la **dirección del flujo de dinero**.  
- **Ratios de Volumen (20 y 50 días)**: relación entre el volumen actual y sus medias móviles (20 y 50). Capturan **picos de actividad** que suelen anticipar movimientos bruscos del precio.  

#### 📝 Ventanas Seleccionadas  

- **Cortas (14–20 días)** → capturan flujos recientes de dinero y cambios rápidos en la presión de compra/venta (MFI 14, CMF 20, ratios de volumen 20).  
- **Medias (50 días)** → aportan contexto de tendencia en el volumen y filtran ruido, útiles para ver si los movimientos de precio tienen un respaldo sostenido.  
- **Indicadores acumulativos (OBV, ADL)** → no dependen de ventanas, pero sus **derivadas y medias móviles** permiten al modelo identificar aceleraciones, divergencias y cruces de tendencia en la presión de volumen.  

👉 En conjunto, estos indicadores permiten evaluar **si el precio sube acompañado de volumen (movimiento sólido)** o si existe una **divergencia (posible reversión)**. Así, enriquecen el dataset con señales adicionales que refuerzan la predicción diaria de la dirección del precio del Bitcoin.  


In [10]:
# =============================================================================
# INDICADORES DE VOLUMEN
# =============================================================================

# ----------------------
# OBV
# ----------------------
obv = ta.volume.OnBalanceVolumeIndicator(
    close=df_binance_1d['Close'], volume=df_binance_1d['Volume']
)
df_binance_1d['OBV'] = obv.on_balance_volume()

# ----------------------
# CMF (Chaikin Money Flow, ventana 20)
# ----------------------
cmf = ta.volume.ChaikinMoneyFlowIndicator(
    high=df_binance_1d['High'], low=df_binance_1d['Low'],
    close=df_binance_1d['Close'], volume=df_binance_1d['Volume'],
    window=20
)
df_binance_1d['CMF_20'] = cmf.chaikin_money_flow()

# ----------------------
# MFI (Money Flow Index, ventana 14)
# ----------------------
mfi = ta.volume.MFIIndicator(
    high=df_binance_1d['High'], low=df_binance_1d['Low'],
    close=df_binance_1d['Close'], volume=df_binance_1d['Volume'],
    window=14
)
df_binance_1d['MFI_14'] = mfi.money_flow_index()

# ----------------------
# A/D Line
# ----------------------
adl = ta.volume.AccDistIndexIndicator(
    high=df_binance_1d['High'], low=df_binance_1d['Low'],
    close=df_binance_1d['Close'], volume=df_binance_1d['Volume']
)
df_binance_1d['ADL'] = adl.acc_dist_index()


# ======================
# Derivadas
# ======================

# ----------------------
# OBV derivadas
# ----------------------
df_binance_1d['OBV_diff'] = df_binance_1d['OBV'].diff()
df_binance_1d['OBV_roc_5'] = df_binance_1d['OBV'].pct_change(5)
df_binance_1d['OBV_ma20'] = df_binance_1d['OBV'].rolling(20).mean()
df_binance_1d['OBV_ma_cross'] = df_binance_1d['OBV'] - df_binance_1d['OBV_ma20']
 
# ----------------------
# CMF derivadas
# ----------------------
df_binance_1d['CMF_diff'] = df_binance_1d['CMF_20'].diff()
df_binance_1d['CMF_zscore'] = (
    (df_binance_1d['CMF_20'] - df_binance_1d['CMF_20'].rolling(20).mean()) /
    df_binance_1d['CMF_20'].rolling(20).std()
)

# ----------------------
# MFI derivadas
# ----------------------
df_binance_1d['MFI_diff'] = df_binance_1d['MFI_14'].diff()
df_binance_1d['MFI_zscore'] = (
    (df_binance_1d['MFI_14'] - df_binance_1d['MFI_14'].rolling(20).mean()) /
    df_binance_1d['MFI_14'].rolling(20).std()
)

# ----------------------
# A/D Line derivadas
# ----------------------
df_binance_1d['ADL_diff'] = df_binance_1d['ADL'].diff()
df_binance_1d['ADL_roc_5'] = df_binance_1d['ADL'].pct_change(5)

# ----------------------
# Volumen ratios
# ----------------------
df_binance_1d['Vol_SMA20'] = df_binance_1d['Volume'].rolling(20).mean()
df_binance_1d['Vol_SMA50'] = df_binance_1d['Volume'].rolling(50).mean()
df_binance_1d['Vol_ratio_20'] = df_binance_1d['Volume'] / df_binance_1d['Vol_SMA20']
df_binance_1d['Vol_ratio_50'] = df_binance_1d['Volume'] / df_binance_1d['Vol_SMA50']

df_binance_1d

,Open time,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,...,CMF_diff,CMF_zscore,MFI_diff,MFI_zscore,ADL_diff,ADL_roc_5,Vol_SMA20,Vol_SMA50,Vol_ratio_20,Vol_ratio_50
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,3.454770e+06,3427.0,616.248541,2.678216e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,5.086958e+06,5233.0,972.868710,4.129123e+06,...,NaN,NaN,NaN,NaN,-259.386591,NaN,NaN,NaN,NaN,NaN
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,1.549484e+06,2153.0,274.336042,1.118002e+06,...,NaN,NaN,NaN,NaN,279.434239,NaN,NaN,NaN,NaN,NaN
3,2017-08-20,4139.98,4211.08,4032.62,4086.29,467.083022,1.930364e+06,2321.0,376.795947,1.557401e+06,...,NaN,NaN,NaN,NaN,-186.142242,NaN,NaN,NaN,NaN,NaN
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,2.797232e+06,3972.0,557.356107,2.255663e+06,...,NaN,NaN,NaN,NaN,1.963761,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2921,2025-08-16,117342.04,117898.99,117143.98,117380.66,6393.681170,7.517194e+08,1179842.0,2995.228650,3.521588e+08,...,-0.028431,-1.545400,1.392974,1.621417,-2385.107881,0.000316,14915.737933,14814.381226,0.428653,0.431586
2922,2025-08-17,117380.66,118575.00,117172.21,117405.01,5898.641920,6.956292e+08,1177563.0,2804.731130,3.307994e+08,...,0.006601,-1.320611,-0.238007,1.431524,-3940.823802,-0.001067,14513.049263,14866.710594,0.406437,0.396768
2923,2025-08-18,117405.01,117543.75,114640.14,116227.05,17741.469250,2.053300e+09,3345487.0,7647.218200,8.850528e+08,...,0.013452,-1.011323,-6.913866,0.918923,1651.021455,-0.002474,14643.226003,15084.905251,1.211582,1.176107
2924,2025-08-19,116227.05,116725.69,112732.58,112872.94,18065.420860,2.065005e+09,3291170.0,8609.360780,9.840874e+08,...,-0.075620,-1.857610,-1.490875,0.752750,-16795.402017,-0.002272,14767.288534,15251.131170,1.223340,1.184530


### 🧹 Limpieza de filas con NaN tras calcular indicadores técnicos

Al calcular indicadores técnicos como SMA, EMA, RSI, ATR o MACD, las primeras filas del DataFrame suelen quedar con valores `NaN`. Esto ocurre porque estos indicadores requieren un número mínimo de datos para su cálculo (por ejemplo, una SMA de 20 días necesita los primeros 20 cierres para dar un valor).  

Para preparar los datos para la posterior fase de modelización, es recomendable eliminar estas filas iniciales. De esta forma, evitamos introducir ruido o sesgo a los modelos.


In [11]:
# =============================================================================
# OBTENER COLUMNAS DE INDICADORES TECNICOS
# =============================================================================

# Obtener todas las columnas
all_columns = df_binance_1d.columns.tolist()

# Índice de 'target'
start_idx = all_columns.index('Target') + 1  # +1 para empezar después de 'target'

# Columnas de indicadores
indicator_columns = all_columns[start_idx:]
print(indicator_columns)

['SMA_20', 'Close_SMA_ratio_20', 'SMA_50', 'Close_SMA_ratio_50', 'EMA_20', 'Close_EMA_ratio_20', 'EMA_50', 'Close_EMA_ratio_50', 'MACD', 'MACD_signal', 'MACD_diff', 'PSAR', 'KAMA_10', 'Close_KAMA_ratio', 'RSI_7', 'RSI_14', 'WR_7', 'WR_14', 'STOCH_K_7', 'STOCH_D_7', 'STOCH_K_14', 'STOCH_D_14', 'ROC_5', 'ROC_10', 'ROC_14', 'MOM_5', 'MOM_10', 'dRSI_7', 'dRSI_14', 'dROC_5', 'dROC_10', 'dROC_14', 'dMOM_5', 'dMOM_10', 'BB_high_10', 'BB_low_10', 'BB_mid_10', 'BB_width_10', 'BB_percent_10', 'BB_high_20', 'BB_low_20', 'BB_mid_20', 'BB_width_20', 'BB_percent_20', 'ATR_14', 'ATR_20', 'KC_high_10', 'KC_low_10', 'KC_mid_10', 'KC_width_10', 'KC_high_20', 'KC_low_20', 'KC_mid_20', 'KC_width_20', 'DC_high_20', 'DC_low_20', 'DC_mid_20', 'DC_width_20', 'OBV', 'CMF_20', 'MFI_14', 'ADL', 'OBV_diff', 'OBV_roc_5', 'OBV_ma20', 'OBV_ma_cross', 'CMF_diff', 'CMF_zscore', 'MFI_diff', 'MFI_zscore', 'ADL_diff', 'ADL_roc_5', 'Vol_SMA20', 'Vol_SMA50', 'Vol_ratio_20', 'Vol_ratio_50']


In [12]:
# =============================================================================
# LIMPIEZA DE FILAS CON NAN
# =============================================================================

df_binance_1d = df_binance_1d.dropna(subset=indicator_columns).reset_index(drop=True)
df_binance_1d

,Open time,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,...,CMF_diff,CMF_zscore,MFI_diff,MFI_zscore,ADL_diff,ADL_roc_5,Vol_SMA20,Vol_SMA50,Vol_ratio_20,Vol_ratio_50
0,2017-10-05,4208.59,4355.00,4110.00,4292.43,779.138638,3.295533e+06,9158.0,351.042019,1.483037e+06,...,-0.049310,0.885758,7.496538,1.470392,381.173703,-0.040948,852.802411,835.249547,0.913622,0.932821
1,2017-10-06,4318.99,4417.00,4292.00,4369.00,506.529176,2.212035e+06,6546.0,226.148177,9.881066e+05,...,0.009643,1.089518,5.823683,1.932982,117.514769,-0.131360,813.250672,829.477123,0.622845,0.610661
2,2017-10-07,4369.00,4479.50,4312.56,4423.00,297.597500,1.302533e+06,4804.0,145.313076,6.371469e+05,...,-0.018074,0.731395,-1.268533,1.583199,96.156758,0.036745,794.021986,811.431308,0.374798,0.366756
3,2017-10-08,4425.00,4658.00,4425.00,4640.00,518.462004,2.347356e+06,7580.0,280.094854,1.268661e+06,...,-0.008783,0.588796,5.445104,1.897959,438.356287,0.209126,768.444764,814.174353,0.674690,0.636795
4,2017-10-09,4640.00,4889.98,4550.00,4786.95,646.463145,3.040509e+06,10372.0,350.756559,1.654275e+06,...,0.041166,1.373108,0.198320,1.666775,254.645404,0.457618,755.651315,817.761955,0.855505,0.790527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2872,2025-08-16,117342.04,117898.99,117143.98,117380.66,6393.681170,7.517194e+08,1179842.0,2995.228650,3.521588e+08,...,-0.028431,-1.545400,1.392974,1.621417,-2385.107881,0.000316,14915.737933,14814.381226,0.428653,0.431586
2873,2025-08-17,117380.66,118575.00,117172.21,117405.01,5898.641920,6.956292e+08,1177563.0,2804.731130,3.307994e+08,...,0.006601,-1.320611,-0.238007,1.431524,-3940.823802,-0.001067,14513.049263,14866.710594,0.406437,0.396768
2874,2025-08-18,117405.01,117543.75,114640.14,116227.05,17741.469250,2.053300e+09,3345487.0,7647.218200,8.850528e+08,...,0.013452,-1.011323,-6.913866,0.918923,1651.021455,-0.002474,14643.226003,15084.905251,1.211582,1.176107
2875,2025-08-19,116227.05,116725.69,112732.58,112872.94,18065.420860,2.065005e+09,3291170.0,8609.360780,9.840874e+08,...,-0.075620,-1.857610,-1.490875,0.752750,-16795.402017,-0.002272,14767.288534,15251.131170,1.223340,1.184530


# 🔹 Métricas On-Chain

Para la obtención de las métricas **on-chain** de la red Bitcoin se ha utilizado un **scraper** desarrollado en el notebook [scraper-blockchain](https://github.com/misanchz98/bitcoin-direction-prediction/blob/main/data/scraper-blockchain-data.ipynb).  Este script obtiene datos directamente desde **Blockchain.com**, lo que permite examinar el funcionamiento de la red y generar métricas clave para su análisis.  

A continuación, se detallan las métricas consideradas:

- `mempool_size`: Tamaño agregado (en bytes) de todas las transacciones que se encuentran pendientes de confirmación en el mempool. 
- `transaction_rate`: Número de transacciones añadidas al mempool por segundo.  
- `average_block_size`: Tamaño promedio de los bloques en la blockchain, medido en megabytes (MB).  
- `average_confirmation_time`: Tiempo promedio que tarda una transacción en ser incluida en un bloque y confirmada dentro de la blockchain.  
- `market_cap_usd`: Valor total de la oferta circulante de Bitcoin, calculado como el promedio diario del precio de mercado en los principales exchanges.  
- `market_price_usd`: Precio promedio en USD del Bitcoin en los principales exchanges.  
- `exchange_volume_usd`: Volumen total de transacciones (en USD) realizadas en los principales exchanges.  
- `hash_rate`: Estimación de la potencia de cómputo de la red, medida en **tera hashes por segundo (TH/s)**.  
- `difficulty`: Medida relativa de la dificultad para encontrar un nuevo bloque. Se ajusta periódicamente en función del poder de cómputo disponible en la red.  
- `miners_revenue`: Ingresos totales percibidos por los mineros, provenientes de recompensas de bloque (coinbase) y tarifas de transacción.  
- `total_transaction_fees`: Valor total de todas las tarifas de transacción pagadas a los mineros, excluyendo la recompensa por bloque. 

In [13]:
# =============================================================================
# LECTURA CSV BLOCKCHAIN DIARIO
# =============================================================================

# Ruta al archivo
file_path = BASE_DIR / BLOCKCHAIN_PATH / BLOCKCHAIN_DAILY_CSV

# Convertir a string con barras normales
file_path_str = str(file_path).replace("\\", "/")

# Leemos fichero
df_blockchain_1d = pd.read_csv(file_path_str)

df_blockchain_1d

,Open time,mempool_size,transaction_rate,market_cap_usd,average_block_size,market_price_usd,exchange_volume_usd,average_confirmation_time,hash_rate,difficulty,miners_revenue,total_transaction_fees
0,2009-01-03,0.00,0.000000,0.000000e+00,0.000000,0.00,0.000000e+00,0.000000,4.971027e-08,1.000000e+00,0.000000e+00,0.000000
1,2009-01-04,0.00,0.000000,0.000000e+00,0.000000,0.00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
2,2009-01-05,0.00,0.000000,0.000000e+00,0.000000,0.00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
3,2009-01-06,0.00,0.000000,0.000000e+00,0.000000,0.00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
4,2009-01-07,0.00,0.000000,0.000000e+00,0.000000,0.00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
6069,2025-08-16,771281.50,2.975000,2.344190e+12,1.665970,117419.50,3.855598e+08,8.231568,9.780076e+08,1.294352e+14,5.629125e+07,2.989912
6070,2025-08-17,562122.00,2.658333,2.352079e+12,1.523959,117484.12,1.547407e+08,5.932923,9.780076e+08,1.294352e+14,5.711806e+07,2.540358
6071,2025-08-18,609931.00,3.316667,2.305944e+12,1.562829,117455.50,1.693708e+08,10.626850,9.908761e+08,1.294352e+14,5.770019e+07,3.392547
6072,2025-08-19,910637.75,3.283333,2.294257e+12,1.739986,116251.12,4.406906e+08,12.908552,8.621909e+08,1.294352e+14,4.993566e+07,3.270740


In [14]:
# =============================================================================
# COMBINAMOS DATOS DE BINANCE CON BLOCKCHAIN
# =============================================================================

# Convertimos columna temporal en formato datetime
df_binance_1d['Open time'] = pd.to_datetime(df_binance_1d['Open time'], format='%Y-%m-%d')
df_blockchain_1d['Open time'] = pd.to_datetime(df_blockchain_1d['Open time'], format='%Y-%m-%d')

# Obtener la fecha mínima y máxima de df_bitcoin_1d
start_date = df_binance_1d['Open time'].min()
end_date = df_binance_1d['Open time'].max()

# Filtrar df_blockchain_1d para que solo contenga fechas dentro del rango
df_blockchain_1d = df_blockchain_1d[
    (df_blockchain_1d['Open time'] >= start_date) & (df_blockchain_1d['Open time'] <= end_date)
]

# Merge por Open time
df_merged = pd.merge(df_binance_1d, df_blockchain_1d, on='Open time', how='inner')

# Mostrar el resultado
df_merged

,Open time,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,...,transaction_rate,market_cap_usd,average_block_size,market_price_usd,exchange_volume_usd,average_confirmation_time,hash_rate,difficulty,miners_revenue,total_transaction_fees
0,2017-10-05,4208.59,4355.00,4110.00,4292.43,779.138638,3.295533e+06,9158.0,351.042019,1.483037e+06,...,2.791667,7.120069e+10,1.018034,4211.55,2.127088e+08,111.082550,6.983443e+06,1.123863e+12,7.562751e+06,160.344240
1,2017-10-06,4318.99,4417.00,4292.00,4369.00,506.529176,2.212035e+06,6546.0,226.148177,9.881066e+05,...,2.866667,7.255732e+10,0.920436,4318.58,2.057251e+08,31.514477,8.882940e+06,1.123863e+12,9.444311e+06,149.021202
2,2017-10-07,4369.00,4479.50,4312.56,4423.00,297.597500,1.302533e+06,4804.0,145.313076,6.371469e+05,...,2.550000,7.216553e+10,0.836508,4369.57,1.915422e+08,20.892141,8.827072e+06,1.123863e+12,9.178387e+06,108.215564
3,2017-10-08,4425.00,4658.00,4425.00,4640.00,518.462004,2.347356e+06,7580.0,280.094854,1.268661e+06,...,2.666667,7.419533e+10,0.764420,4439.46,1.281929e+08,20.239779,1.016789e+07,1.123863e+12,1.098203e+07,111.796475
4,2017-10-09,4640.00,4889.98,4550.00,4786.95,646.463145,3.040509e+06,10372.0,350.756559,1.654275e+06,...,3.283333,7.648983e+10,0.921536,4605.66,2.516054e+08,23.698470,8.994675e+06,1.123863e+12,1.016728e+07,149.340987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2872,2025-08-16,117342.04,117898.99,117143.98,117380.66,6393.681170,7.517194e+08,1179842.0,2995.228650,3.521588e+08,...,2.975000,2.344190e+12,1.665970,117419.50,3.855598e+08,8.231568,9.780076e+08,1.294352e+14,5.629125e+07,2.989912
2873,2025-08-17,117380.66,118575.00,117172.21,117405.01,5898.641920,6.956292e+08,1177563.0,2804.731130,3.307994e+08,...,2.658333,2.352079e+12,1.523959,117484.12,1.547407e+08,5.932923,9.780076e+08,1.294352e+14,5.711806e+07,2.540358
2874,2025-08-18,117405.01,117543.75,114640.14,116227.05,17741.469250,2.053300e+09,3345487.0,7647.218200,8.850528e+08,...,3.316667,2.305944e+12,1.562829,117455.50,1.693708e+08,10.626850,9.908761e+08,1.294352e+14,5.770019e+07,3.392547
2875,2025-08-19,116227.05,116725.69,112732.58,112872.94,18065.420860,2.065005e+09,3291170.0,8609.360780,9.840874e+08,...,3.283333,2.294257e+12,1.739986,116251.12,4.406906e+08,12.908552,8.621909e+08,1.294352e+14,4.993566e+07,3.270740


In [15]:
# =============================================================================
# GUARDAMOS RESULTADO
# =============================================================================
df_merged.to_csv('data/btc_historical_data.csv', index=False)